## Agile Hardware Design
***
# Collections

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 293](https://classes.soe.ucsc.edu/cse293/Winter22/)

## Plan for Today

* Scala for & collections (only Seq for today)
* Chisel using collections
* Chisel Vec
* Chisel Mem

## Loading The Chisel Library Into a Notebook

In [ ]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

## Scala `Seq`

* Scala's [collection library](https://docs.scala-lang.org/overviews/collections-2.13/overview.html) is one of its strengths
* [Seq](https://www.scala-lang.org/api/2.12.12/scala/collection/Seq.html) is an ordered collection (sequence), immutable by default
* Index into it with `()`
* A base class with multiple implementations
  * For many of our (small) cases, fine to use directly
  * Some style guides recommend specifity with `List` or `Vector`

In [ ]:
val l = Seq(1,2,3)
l(1)
l.isEmpty
l.nonEmpty
l.length
Seq.fill(3)(8)
l.getClass

## Scala Ranges
* Can instantiate a range by `start until end` (exclusive)
* Can make range inclusive with `start to end`
* Can also provide a custom increment (default is 1) with `by`

In [ ]:
0 until 4
0 to 3
0 to 4 by 2
3 to 0 by -1

## Scala `for`

* Iterates over iterable (typically range or collection)
* Sometimes use with `var` to pass results between iterations
* Also helpful for building up state space for testing
* Can also be used to do full [for comprehensions](https://docs.scala-lang.org/tour/for-comprehensions.html) (nested and filters)

In [ ]:
for (i <- 0 until 4)
    println(i)

// var last = -1
// for (i <- 0 until 4) {
//     println(last)
//     last = i
// }


// for (i <- 0 until 4;
//      j <- i until 4 if (i+j == 4)) {
//     println(i,j)
// }

## Example: Delay Line with `for`

In [ ]:
class DelayNCycles(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val out = Output(Bool())
    })
    require(n > 0)
    val regs = Seq.fill(n)(Reg(Bool()))
    regs(0) := io.in
    for (i <- 1 until n)
        regs(i) := regs(i-1)
    io.out := regs(n-1)
}
println(getVerilog(new DelayNCycles(2)))

<img src="images/delayN.svg" alt="delay line schematic" style="width:45%;" align="right"/>

## Example: Delay Line with `for` and `var`

* Judicious use of `var` can sometimes tidy things up
* Later, will cover functional programming techniques to also clean up

In [ ]:
class DelayNCycles(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val out = Output(Bool())
    })
    var lastConn = io.in
    for (i <- 0 until n)
        lastConn = RegNext(lastConn)
    io.out := lastConn
}
println(getVerilog(new DelayNCycles(5)))

## Using `for` in a Tester

In [ ]:
class CombLogic extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val c   = Input(Bool())
        val out = Output(Bool())
    })
    io.out := (io.a && io.b) || ~io.c
//     printf(p"a: ${io.a}, b: ${io.b}, c: ${io.c}, out: ${io.out}\n")
}

In [ ]:
test(new CombLogic) { dut =>
    for (a <- Seq(true, false)) {
        for (b <- Seq(true, false)) {
            for (c <- Seq(true, false)) {
                dut.io.a.poke(a.B)
                dut.io.b.poke(b.B)
                dut.io.c.poke(c.B)
                println(s"$a, $b, $c")
                val expected = (a && b) || !c
                dut.io.out.expect(expected.B)
//                 dut.clock.step()
            }
        }
    }
}

## Chisel `Vec`

* Chisel collections construct, 2 uses:
  * _Dynamic addressing_ in hardware (muxes)
  * Paramterize number of ports
* `Vec(num_elements, type)`
* Most often will want a Scala collection for addressing during elaboration
* Use `Reg` of `Vec` for state
  * `Vec` of `Reg` not possible

```scala
Reg(Vec(num_elements, type))
```
* Can also use with `Wire`

```scala
Wire(Vec(num_elements, type))
```

In [ ]:
class MyMuxN(n: Int, m: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Vec(n, UInt(m.W)))
        val sel = Input(UInt(log2Ceil(n).W))
        val out = Output(UInt(m.W))
    })
    io.out := io.in(io.sel)
}
println(getVerilog(new MyMuxN(4,1)))

## Using `Vec` to Parameterize # of Ports

In [ ]:
class Reducer(n: Int, m: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Vec(n, UInt(m.W)))
//         val in  = Input(UInt((n*m).W))
        val out = Output(UInt(m.W))
    })
    require(n > 0)
    var totalSoFar = io.in(0)
    for (i <- 1 until n)
        totalSoFar = io.in(i) + totalSoFar
    io.out := totalSoFar
}
println(getVerilog(new Reducer(2,2)))

<img src="images/reducer.svg" alt="reducer schematic" style="width:65%;" align="right"/>

## Read-Only Memory (ROM) with `VecInit`
* `VecInit` will create a `Wire` with its input
* Can also use `VecInit` to initialize registers (with `RegInit`)

In [ ]:
class DivByXTable(x: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(4.W))
        val out = Output(Bool())
    })
    var results = Seq[Bool]()
    for (i <- 0 to 15) {
        results = results :+ (i % x == 0).B
    }
    val table = VecInit(results)
    io.out := table(io.in)
}

In [ ]:
println(getVerilog(new DivByXTable(3)))

## Chisel `Mem`

* Construct for [memory](https://www.chisel-lang.org/chisel3/docs/explanations/memories.html) (dynamically addressable & mutable)
* Backend will choose appropriate implementation technology
* Default (Mem): _combinational read_ (0 cycle delay), _synchronous write_ (1 cycle delay)
  * Can tweak delay parameters
  * `SyncReadMem` has 1 cycle read delay
* Memory ports can be declared implicitly or explicitly
* Also has support for write masks

## Example: Register File (2R, 1W) - implicit ports

In [ ]:
class RegFile(nRead: Int) extends Module {
    val io = IO(new Bundle {
        val r0addr = Input(UInt(5.W))
        val r1addr = Input(UInt(5.W))
        val w0addr = Input(UInt(5.W))
        val w0en =   Input(Bool())
        val w0data = Input(UInt(64.W))
        val r0out =  Output(UInt(64.W))
        val r1out =  Output(UInt(64.W))
    })
//     val regs = Mem(32, UInt(64.W))
    val regs = Reg(Vec(32, UInt(64.W)))
    io.r0out := regs(io.r0addr)
    io.r1out := regs(io.r1addr)
    when(io.w0en) {
        regs(io.w0addr) := io.w0data
    }
}

In [ ]:
println(getVerilog(new RegFile(8)))

## Example: Register File (2R, 1W) - explicit ports

In [ ]:
class RegFile extends Module {
    val io = IO(new Bundle {
        val r0addr = Input(UInt(5.W))
        val r1addr = Input(UInt(5.W))
        val w0addr = Input(UInt(5.W))
        val w0en =   Input(Bool())
        val w0data = Input(UInt(64.W))
        val r0out =  Output(UInt(64.W))
        val r1out =  Output(UInt(64.W))
    })
    val regs = Mem(32, UInt(64.W))
    io.r0out := regs.read(io.r0addr)
    io.r1out := regs.read(io.r1addr)
    when (io.w0en) {
        regs.write(io.w0addr,io.w0data)
    }
}

In [ ]:
println(getVerilog(new RegFile()))

## Example: Register File (NR, 1W) - parameterized ports

In [ ]:
class RegFile(nRead: Int) extends Module {
    val io = IO(new Bundle {
        val raddr  = Input(Vec(nRead, UInt(5.W)))
        val w0addr = Input(UInt(5.W))
        val w0en =   Input(Bool())
        val w0data = Input(UInt(64.W))
        val rout = Output(Vec(nRead, UInt(64.W)))
    })
    val regs = Mem(32, UInt(64.W))
    for (i <- 0 until nRead)
        io.rout(i) := regs(io.raddr(i))
    when(io.w0en) {
        regs(io.w0addr) := io.w0data
    }
}

In [ ]:
println(getVerilog(new RegFile(4)))

### Summary - When to Use Scala or Chisel Collection?

* You will want to use a Scala collection (probably `Seq`) most often
    * In a generator, want to instantiate N things
    * You may need to address which thing you want, but you are accessing it during hardware generation time
* You will need Chisel collections (`Vec`, `Mem`) when ...
    * You want the generated hardware to dynamically address components (put muxes into actual hardware)
    * You definitely want a memory (e.g. `SyncReadMem`)